In [ ]:
# test run
import openai
import re
from openai._client import OpenAI
client = OpenAI()

import requests
import json

client = OpenAI()

def clean(query):

  # regex to clean the input string
  # Remove non-alphanumeric characters except spaces and periods
  clean_query = re.sub(r"[^a-zA-Z0-9\s.]", "", query)


  # Check whether the prompt is complete - validation layer
  # Basic check for length and content
  if len(clean_query) < 5 or clean_query.strip(".") == "":
      return "Try Again. Please provide more details about your complaint."

  # reprompt using LLM itself
  # If input passes basic validation but still needs clarification

  instruction = f"""
  Rewrite the user's input for filing a complaint so that it is clear and specific. If it is incomplete or blank, output 'Try Again'
  """

  messages = [{"role": "system", "content": instruction},
              # {"role": "assistant", "content": "What is your complaint? Also mention the locality from where the complaint is being filed."},
              {"role": "user", "content": clean_query}
            ]
  response = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=messages,
  )
  response_message = response.choices[0].message

  # Check if LLM response is asking for a retry
  if "Try Again" in response_message:
      return "Try Again. Please provide a clearer description of your complaint."

  return response_message

def run_conversation():
    query = input("Hi Active Citizen, please write your complaint.")
    complaint = clean(query)

    category_list = ['StreetLights', 'Garbage', 'Drains'] # can add all 19 unique categories
    categories = ', '.join(category_list)

    # using few shot prompting
    instruction = f"""
    You are a helpful assitant who helps users file complaints. Read the following user complaint and classify it into one of the following categories:
    {categories}
    Complaint: {complaint}
    Category: <Output the correct Category here>

    ### Below are some examples of user complaints and their correct classifications:

    Example 1:
    Complaint: 'Excessive garbage burning in our city poses a major health threat. Illegal fires emit toxic smoke, worsening air quality. Urgent action is needed to remove and manage plastic waste.'
    Classification: Garbage

    Example 2:
    Complaint: 'There's a broken streetlight on Main Street near the library. It's been dark for a few weeks now, and it feels unsafe to walk there at night. Can this be fixed soon?'
    Classification: StreetLights

    Example 3:
    Complaint: 'The drainage system on Elm Street is clogged. During the last rain, water overflowed and flooded the basements of several houses. The drains need to be cleaned to prevent further damage.'
    Classification: Drains

    """

    messages = [{"role": "system", "content": instruction},
                {"role": "assistant", "content": "What is your complaint? Also mention the locality from where the complaint is being filed."},
                {"role": "user", "content": complaint}
              ]
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        #tools=tools,
        #tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message

    return response_message

In [ ]:
run_conversation()

Hi Active Citizen, please write your complaint. Attach images if you have one
My streets are filled with trash cans and household waste


ChatCompletionMessage(content='Category: Garbage', role='assistant', function_call=None, tool_calls=None)

In [ ]:
run_conversation()

Hi Active Citizen, please write your complaint. Attach images if you have one
My drain has a lot of garbage in it 


ChatCompletionMessage(content='Category: Drains', role='assistant', function_call=None, tool_calls=None)

In [ ]:
run_conversation()

Hi Active Citizen, please write your complaint.


ChatCompletionMessage(content='Category: Unable to classify (the provided text is a prompt for more information rather than a complaint with sufficient details to classify)', role='assistant', function_call=None, tool_calls=None)